In [ ]:
# !pip install psycopg2

   ---------------------------------------- 0.0/1.2 MB ? eta -:--:--
   --------- ------------------------------ 0.3/1.2 MB ? eta -:--:--
   ------------------------------------ --- 1.0/1.2 MB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 1.2/1.2 MB 3.2 MB/s eta 0:00:00


In [ ]:
# !pip install asyncpg

   ---------------------------------------- 0.0/625.8 kB ? eta -:--:--
   ---------------------------------------- 625.8/625.8 kB 7.7 MB/s eta 0:00:00


In [ ]:
# !pip install passlib

   ---------------------------------------- 0.0/525.6 kB ? eta -:--:--
   ---------------------------------------  524.3/525.6 kB 3.4 MB/s eta 0:00:01
   ---------------------------------------- 525.6/525.6 kB 2.8 MB/s eta 0:00:00


In [ ]:
# !pip install bcrypt

  Using cached bcrypt-4.2.1-cp39-abi3-win_amd64.whl.metadata (10 kB)
Using cached bcrypt-4.2.1-cp39-abi3-win_amd64.whl (153 kB)


In [13]:
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="sql_project_p3",
    user="postgres",
    password="admin123"
)


In [ ]:
# # Create a cursor object to execute SQL queries
# cursor = conn.cursor()

# # Execute a sample query
# cursor.execute("SELECT emp_id, user_name, password_hash FROM employees;")

# # Fetch all rows
# rows = cursor.fetchall()

# # Print data
# for row in rows:
#     print(row)

# # Close connection
# cursor.close()
# conn.close()

In [1]:
from passlib.context import CryptContext
import psycopg2

# Connect to PostgreSQL
conn = psycopg2.connect(
    host="localhost",
    database="sql_project_p3",
    user="postgres",
    password="admin123"
)
# Setup password hashing
pwd_context = CryptContext(schemes=["bcrypt"], deprecated="auto")

cursor = conn.cursor()

# Employee credentials
employees = [
    ('E101', 'johe101', 'joh101'),
    ('E102', 'jane102', 'jan102'),
    ('E103', 'mike103', 'mik103'),
    ('E104', 'emie104', 'emi104'),
    ('E105', 'sare105', 'sar105'),
    ('E106', 'mice106', 'mic106'),
    ('E107', 'mice107', 'mic107'),
    ('E108', 'jese108', 'jes108'),
    ('E109', 'dane109', 'dan109'),
    ('E110', 'laue110', 'lau110'),
    ('E111', 'chre111', 'chr111')
]

# Insert hashed passwords
for emp_id, username, password in employees:
    hashed_password = pwd_context.hash(password)
    cursor.execute("UPDATE employees SET user_name = %s, password_hash = %s WHERE emp_id = %s", 
                   (username, hashed_password, emp_id))

conn.commit()
cursor.close()
conn.close()


(trapped) error reading bcrypt version
Traceback (most recent call last):
  File "c:\Users\kmnik\anaconda3\envs\env_auc\lib\site-packages\passlib\handlers\bcrypt.py", line 620, in _load_backend_mixin
    version = _bcrypt.__about__.__version__
AttributeError: module 'bcrypt' has no attribute '__about__'
